# HW6:  60 points possible

This assignment needs the file books2.csv, which should be at the same location where you got this file.

# Problem 1:  The Book Class (54 points:  3,5,5,5,6,8,10,7,5)

a) Create a Book class with the following methods:

* (3 pts) A constructor that takes the author (string), title (string), and a list of ratings (integers)
* (5 pts) A `__str__()` method that returns the string "[title] by [author]" when invoked
* (5 pts) An `average_rating()` method that takes no argument and returns the average of all the ratings in the list
* (5 pts) A `rate()` method that takes one integer argument and adds that integer to the end of the ratings, returning the new list of ratings

In [84]:
# TODO Book class with __init__(), __str__(), average_rating(), rate()
class Book:
  def __init__(self,author,title,ratings):
    self.title = title
    self.author = author
    self.ratings = ratings

  def __str__(self):
    return str(self.title) + ' by ' + str(self.author)
  
  def average_rating(self):
    return (sum(self.ratings))/(len(self.ratings))
  
  def rate(self,integer):
    self.ratings.append(integer)
    return self.ratings


In [85]:
# Tests
my_book = Book('Mary Shelley', 'Frankenstein', [5, 4, 3, 5, 3])
print(my_book) # expect 'Frankenstein by Mary Shelley'
print(my_book.average_rating()) # expect 4.0
print(my_book.rate(5)) # expect [5, 4, 3, 5, 3, 5]

Frankenstein by Mary Shelley
4.0
[5, 4, 3, 5, 3, 5]


b, 6 pts) Sometimes data for an entity can be spread across multiple records in the dataset.  Code the function merge() which takes two Book objects as arguments, and returns a single Book record where the author and title match the first argument, and the ratings are a concatenation of the two ratings lists.

In [86]:
# TODO merge()
def merge(book1,book2):
  books = book1.ratings+book2.ratings
  return Book(book1.author,book1.title,books)

In [87]:
book1 = Book('Jane Austen', 'Pride and Prejudice', [5,4,5,3])
book2 = Book('J. Austen', 'Pride and Prejudice', [5, 5])
book3 = merge(book1,book2)
print(book3)
print(book3.ratings)

Pride and Prejudice by Jane Austen
[5, 4, 5, 3, 5, 5]


c, 8 pts) Code a class that inherits from Book, Nonfiction.  Give it a constructor identical to Book's, except it has one additional argument:  the edition number (an int).  Then add to the Nonfiction class a method more_recent_than() which takes a Book as an argument and returns true if 1) the authors and titles match, *and* 2) the Book calling it has a higher edition number than the argument.

In [88]:
# TODO Nonfiction class with __init__(), more_recent_than()
class Nonfiction(Book):
  def __init__(self,title,author,ratings,edition):
    Book.__init__(self,title,author,ratings)
    self.edition = edition
    
  def more_recent_than(self,book):
    if self.author == book.author and self.title == book.title and book.edition < self.edition:
      return True
    else:
      return False

In [89]:
book1 = Nonfiction('Feynman', 'Physics', [5,4,5], 1)
book2 = Nonfiction('Feynman', 'Physics', [5,5,5], 2)
book3 = Nonfiction('Herschbach', 'Chemistry', [5,5,5], 1)

print(book2.more_recent_than(book1)) # expect True
print(book2.more_recent_than(book3)) # expect False, author/title mismatch
print(book3.average_rating()) # Should have inherited this method from Book

True
False
5.0


d, 10pts) Code a class Fiction that inherits from Book and has an extra attribute *sequel*, which should be a Fiction (or None if there is no sequel).  Accept an extra argument in the constructor to set this attribute.  Throw a ValueError if the sequel argument isn't a Fiction object or None.

Then write a method rest_of_series() that returns a list of the remaining titles in the series (as strings).  For example, if the Book The Fellowship of the Ring had the sequel The Two Towers, and that had a sequel of The Return of the King, then calling this method of The Fellowship of the Ring would return the list of strings ['The Two Towers', 'The Return of the King'].  (Your method should work with arbitrarily long chains of sequels, not just trilogies.)

In [90]:
# TODO Fiction class with __init__(), rest_of_series()
class Fiction(Book):
  def __init__(self,author,title,rating,sequel):
    super().__init__(author,title,rating)
    self.sequel = sequel
    try:
      sequel.edition
      raise ValueError('The sequel argument is not a Fiction object or None')
    except AttributeError:
      return None
  def rest_of_series(self):
    sequellist = []
    if self.sequel != None:
      sequellist.append(self.sequel.title)
      sequellist.extend(self.sequel.rest_of_series())
    return sequellist

In [91]:
# Test, should throw an error because sequel isn't Fiction
book1 = Nonfiction('Feynman', 'Physics', [5,4,5], 1)
book2 = Fiction('Mary Shelley', 'Frankenstein', [5,5,5], book1)

ValueError: ignored

In [92]:
# Test rest_of_series()
king = Fiction('J.R.R. Tolkien', 'The Return of the King', [], None)
towers = Fiction('J.R.R. Tolkien', 'The Two Towers', [], king)
fellowship = Fiction('J.R.R. Tolkien', 'The Fellowship of the Ring', [], towers)
hobbit = Fiction('J.R.R. Tolkien', 'The Hobbit', [], fellowship)

print(hobbit.rest_of_series()) # Should be ['The Fellowship of the Ring', 'The Two Towers', 'The Return of the King']
print(king.rest_of_series()) # Expect empty list []

['The Fellowship of the Ring', 'The Two Towers', 'The Return of the King']
[]


e, 7 pts)  Create a new BookDict class.  The argument for the constructor should be the string name of a csv file.  The csv columns in order from left to right are title, author, and a comma-separated string of ratings.  The constructor should create a dictionary from title (string) to the Book object with that title.  Call that dictionary my_dict and make it a field (attribute) of the BookDict object.  (You can assume titles are unique for this exercise.  Note that you will need to use the function float() to convert string ratings to floating point numbers.)

f, 5 pts) The one other method for the BookDict, lookup(), should take a title (string) as an argument, and print the information stored in the Book for that title.  The printed information should follow the format '[title] by [author] (average_rating)' -- for example, 'Pride and Prejudice by Jane Austen (4.2)'.  Print no more than two digits after the decimal place.  lookup() should also return the Book, besides printing its information.  If the title isn't in the dictionary, print 'Not Found' and return None.  Use methods you have already created when you can.  You can again assume titles are unique.

In [98]:
import csv
import numpy as np

class BookDict:
  def __init__(self,filename):
    self.my_dict = {}
    with open(filename,'r') as myfile:
      reader = csv.reader(myfile)
      for element in reader:
        title = element[0]
        author = element[1]
        rating = list(map(float,element[2].split(',')))
        book = Book(author,title,rating)
        self.my_dict[title] = book

  def lookup(self,title):
    if title in self.my_dict:
      book = self.my_dict[title]
      avg_rating = book.average_rating()
      print(f'{title} by {book.author} ({avg_rating:.2f})')
      return book
    else:
      print('Not Found')
      return None

In [99]:
# Test:  should print 'Pride and Prejudice by Jane Austen (4.2)'
my_bookdict = BookDict('books2.csv')
my_book = my_bookdict.lookup('Pride and Prejudice')
# Test:  should print 'Not Found'
book2 = my_bookdict.lookup('Erehwon')

Pride and Prejudice by Jane Austen (4.20)
Not Found


In [100]:
from google.colab import files
files.upload() # Upload books2.csv

Saving books2.csv to books2 (2).csv


{'books2.csv': b'Pride and Prejudice,Jane Austen,"4,4,4,4,5"\r\nSense and Sensibility,Jane Austen,"3,4,3,4,5,4,4"\r\n1Q84,Haruki Murakami,"4,2,5,3,4,3,3,5"\r\n"Little, Big",John Crowley,"4,3,5,4,3,4,5,5"\r\nPossession,A.S. Byatt,"3,5,4,5,4,5,5"\r\nDragons of Autumn Twilight,Margaret Weis and Tracy Hickman,"2,5,1,4,3,3,4,4,2"\r\nUnder the Whispering Door,T.J. Klune,"4,5,5,4,3,3,4"\r\nThe Hitchhiker\xe2\x80\x99s Guide To the Galaxy,Douglas Adams,"5,3,5,4,5,5,2,5"\r\n'}

# Problem 2:  Recursion (6 points)

Write a *recursive* function that calculates $f(n)$ where $f(n) = f(n-1) + f(n-3) + 1$ and $f(1) = 1, f(2) = 2, f(3) = 3$.  You can assume $n$ is a positive integer.

In [102]:
# TODO f(n)
def f(n):
  if n == 1:
    return 1
  if n == 2:
    return 2
  if n == 3:
    return 3
  return f(n-1)+f(n-3)+1


In [103]:
# Tests
print(f(4)) # Should print 5
print(f(5)) # Should print 8
print(f(6)) # Should print 12

5
8
12
